In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import binom, nbinom
from plotly import graph_objects as go
from plotly.graph_objects import Figure
from summer2 import CompartmentalModel
from summer2.parameters import Parameter, Function

In [ ]:
# Construct a simple model
epimodel = CompartmentalModel(
    [0.0, 100.0],
    ["susceptible", "exposed", "infectious", "recovered"],
    ["infectious"],
)
epimodel.add_infection_frequency_flow(
    "infection",
    Parameter("contact_rate"),
    "susceptible",
    "exposed",
)
epimodel.add_transition_flow(
    "progression",
    Parameter("progression_rate"),
    "exposed",
    "infectious",
)
epimodel.add_transition_flow(
    "recovery",
    Parameter("recovery_rate"),
    "infectious",
    "recovered",
)
init_conds = {"susceptible": Parameter("start_pop") - Parameter("seed"), "infectious": Parameter("seed")}
epimodel.set_initial_population(init_conds)
inc = epimodel.request_output_for_flow("incidence", "progression")
epimodel.request_function_output("notifications", inc * Parameter("cdr"));

In [ ]:
# Arbitrary parameters
params = {
    "contact_rate": 0.8,
    "progression_rate": 0.5,
    "recovery_rate": 0.5,
    "cdr": 0.25,
    "start_pop": 1e4,
    "seed": 1.0,
}

In [ ]:
# Generate synthetic data for calibration (uses CDR)
epimodel.run(params)
outputs = epimodel.get_derived_outputs_df()
synth_target = outputs[::10]["notifications"].astype(int)

### Binomial likelihood
A possible alternative approach.

In [ ]:
# Try out various contact rate parameters
n_params = 10
contact_rates = np.linspace(0.65, 0.95, n_params)
inc_results = pd.DataFrame(columns=contact_rates)
log_likes = np.zeros(n_params)
for c, contact_param in enumerate(contact_rates):

    # Only get the incidence output from the model, not modelled notifications
    epimodel.run(params | {"contact_rate": contact_param})
    model_result = epimodel.get_derived_outputs_df()
    inc_results[contact_param] = model_result["incidence"]
    model_inc_result = model_result.loc[synth_target.index, "incidence"].astype(int)

    # Likelihood calculation then uses the case detection proportion
    likelihood = binom.pmf(synth_target, model_inc_result, params["cdr"]).prod()
    log_like = 1.0 if likelihood == 0.0 else np.log(likelihood)
    print(f"likelihood for contact rate of {round(contact_param, 2)} is {likelihood}")
    log_likes[c] = log_like

In [ ]:
# Try out various contact rate parameters
n_params = 10
contact_rates = np.linspace(0.65, 0.95, n_params)
inc_results = pd.DataFrame(columns=contact_rates)
log_likes = np.zeros(n_params)
for c, contact_param in enumerate(contact_rates):

    # Only get the incidence output from the model, not modelled notifications
    epimodel.run(params | {"contact_rate": contact_param})
    model_result = epimodel.get_derived_outputs_df()
    inc_results[contact_param] = model_result["incidence"]
    model_inc_result = model_result.loc[synth_target.index, "incidence"].astype(int)

    # Likelihood calculation then uses the case detection proportion
    likelihood = binom.pmf(synth_target, model_inc_result, params["cdr"]).prod()
    log_like = 1.0 if likelihood == 0.0 else np.log(likelihood)
    print(f"likelihood for contact rate of {round(contact_param, 2)} is {likelihood}")
    log_likes[c] = log_like

In [ ]:
# Illustrate results
log_likes[log_likes == 1.0] = log_likes.min() - 10.0
fig = Figure()
fig.add_trace(go.Scatter(x=synth_target.index, y=synth_target, name="cases target", mode="markers", line={"color": "black"}))
shades = log_likes / log_likes.min()
for c, contact_param in enumerate(contact_rates):
    shade = round(shades[c] * 100.0)
    fig.add_trace(go.Scatter(x=inc_results[contact_param].index, y=inc_results[contact_param], name=round(contact_param, 2), line={"color": f"hsl(50%,{shade}%,{shade}%)"}))
fig

Note that any model trajectories for which incidence is less than case notifications at a point in time has zero likelihood.
This is arguably better or worse.
It seems more intuitive to me, but I doubt calibration algorithms will be able to handle it,
because no information is provided in the parts of parameter space that are impossible.
In this configuration, I am thinking of what we are modelling as being the true underlying incidence,
because I don't really like this quantity of "modelled notifications".
It seems to imply that we are modelling some epidemic that demonstrably never occurred.
By contrast, here we are modelling an epidemic that could have resulted in the 
profile of case notifications that we saw.

I also note that just scaling the size of the model population while retaining all the relative dynamics
means that the likelihood declines very rapidly to very low numbers.
This makes sense given the properties of the binomial distribution, of course.

I guess we could get around some of these issues with other distributions (e.g. beta), 
but the binomial seems the most suitable to what we're trying to achieve.

### Negative binomial likelihood
Our usual approach.

In [ ]:
# Equivalent process using the negative binomial distribution as per our usual approach
case_results = pd.DataFrame(columns=contact_rates)
log_likes = np.zeros(n_params)
for c, contact_param in enumerate(contact_rates):

    # Get modelled case notifications
    epimodel.run(params | {"contact_rate": contact_param})
    model_result = epimodel.get_derived_outputs_df()["notifications"]
    case_results[contact_param] = model_result
    model_case_result = model_result[synth_target.index]

    # Calculate likelihood from negative binomial
    dispersion_param = 100.0
    like_components = []
    for i in synth_target.index:
        modelled_val = model_case_result[i]
        p = modelled_val / (modelled_val + dispersion_param)
        like_component = nbinom.pmf(synth_target[i], dispersion_param, 1.0 - p)
        like_components.append(like_component)
    likelihood = np.prod(like_components)

    # Likelihood calculation comparing modelled and observed notifications
    log_like = np.log(likelihood)
    print(f"likelihood for contact rate of {round(contact_param, 2)} is {likelihood}")
    log_likes[c] = log_like

In [ ]:
# Illustrate results
log_likes[log_likes == 1.0] = log_likes.min() - 10.0
fig = Figure()
fig.add_trace(go.Scatter(x=synth_target.index, y=synth_target, name="cases target", mode="markers", line={"color": "black"}))
shades = log_likes / log_likes.min()
for c, contact_param in enumerate(contact_rates):
    shade = round(shades[c] * 100.0)
    fig.add_trace(go.Scatter(x=case_results[contact_param].index, y=case_results[contact_param], name=round(contact_param, 2), line={"color": f"hsl(50%,{shade}%,{shade}%)"}))
fig